In [ ]:
%run optimizers/Optimizer.ipynb

class PoseEstimation(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
    
    def open(self):
        self.session = tf.Session()
        self._initializeVariables()
    
    def close(self):
        self.session.close()
    
    def optimize(self, targetFeatures):
        self._setAComponent()
        self._setHComponent(targetFeatures)
        self.k = tf.linalg.lstsq(
                matrix=self.A,
                rhs=self.h)
        
        k = np.array(self.session.run(self.k))
        
        r1 = np.array([k[0][0], k[1][0], k[2][0]])
        r2 = np.array([k[4][0], k[5][0], k[6][0]])

        s = (np.linalg.norm(r1) + np.linalg.norm(r2)) / 2.
        t = np.array([k[3][0]/s, k[7][0]/s]).reshape((2,1))

        U,S,V = np.linalg.svd(np.array([r1, r2, np.cross(r1, r2)]))
        r = np.matmul(U,V) # TODO, should .T??
        if np.linalg.det(r) == -1:
            U,S,V = np.linalg.svd(np.array([r1, r2, -1*np.cross(r1, r2)]))
            r = np.matmul(U,V.T) # TODO, should .T??
        
        return self.a, r, t, s
    
    def update(self, a, r, t, s):
        self.a = a
        
    def _initializeVariables(self):
        self.A = tf.Variable(np.zeros((2*self.numFeatures, 8)), name='A', trainable=False)
        self.h = tf.Variable(np.zeros((2*self.numFeatures, 1)), name='h', trainable=False)
        ai, ri, ti, si = Optimizer.getDefaults(self.numComponents)
        self.a = ai
    
    def _setAComponent(self):
        modelFeatures = self.mean + np.matmul(self.pcaBasis, self.a)
        A = np.zeros((2*self.numFeatures, 8))
        for i in range(self.numFeatures):
            A[2*i,:]   = [modelFeatures[3*i], modelFeatures[3*i+1], modelFeatures[3*i+2], 1, 0, 0, 0, 0]
            A[2*i+1,:] = [0, 0, 0, 0, modelFeatures[3*i], modelFeatures[3*i+1], modelFeatures[3*i+2], 1]
        self.session.run(self.A.assign(A))
        
    def _setHComponent(self, targetFeatures):
        self.session.run(self.h.assign(np.reshape(targetFeatures, (2*self.numFeatures, 1))))
    